In [1]:
%pip install --upgrade langchain langchain-community langchainhub langchain-openai chromadb unstructured[all-docs] ollama lamini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.5/689.5 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

In [4]:
from dotenv import load_dotenv
import lamini
import os
load_dotenv("/content/drive/MyDrive/DATA298/.env")
lamini.api_key = os.environ["LAMINI_API_KEY"]


In [5]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


- Run Ollama serve in terminal

In [6]:
!ollama pull llama2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifes

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def create_personality_chain(retriever,personality,llm):
  instruction = f"You are a chatbot that communicates like {personality} during an interview. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just answer like {personality} would. Use three sentences maximum and keep the answer concise."
  template = instruction + '''
Context: {context}
Question: {question}
Answer: '''
  prompt = ChatPromptTemplate.from_template(template = template)
  chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    )
  return chain

In [ ]:
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# def create_personality_chain(retriever,personality):
#   instruction = f"You are a chatbot that communicates like {personality} during an interview. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just answer like {personality} would. Use three sentences maximum and keep the answer concise."
#   template = instruction + '''
# Context: {context}
# Question: {question}
# Response:{response} '''
#   prompt = ChatPromptTemplate.from_template(template = template)
#   chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough(), "response": RunnablePassthrough()}
#     | prompt
#     | StrOutputParser()
#     )
#   return chain

In [ ]:
def parse_questions(file_path):
  with open(file_path) as fp:
    for line in fp.readlines():
      yield line.strip()

In [ ]:
def generate_answers(chain,questions_path):
  qa = []
  for question in parse_questions(questions_path):
    answer = chain.invoke(question)
    qa.append({"q":question,"a":answer})
  return qa

In [ ]:
def evaluate_personality_answers(qa,personality):
  template = '''
    You are a evaluator responsible for scoring the quality of the answers from the interview in terms of correctness and style.
    The following answers were generated by a chatbot pretending to be {personality}.
    For each question answer pair, produce only a score from 0 to 1 indicting the quality of the answer
    Question: {question}
    Answer: {answer}
    Score: '''
  prompt = ChatPromptTemplate.from_template(template = template)
  llm = ChatOpenAI(model_name="gpt-4", temperature=0)

  evaluation_chain = (
      prompt
    | llm
    | StrOutputParser()
    )

  scores = []
  for d in qa:
    score = evaluation_chain.invoke({"personality": personality, "question": d["q"], "answer": d["a"]})
    scores.append(float(score))
  avg = sum(scores)/len(scores)
  return avg

In [ ]:
# Load, chunk and index the contents of the blog.
loader = UnstructuredFileLoader("elonmusk-ft.txt")

docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
elon_db = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
retriever = elon_db.as_retriever()

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

elon_gpt35_chain = create_personality_chain(retriever,"elon musk",llm)

In [ ]:
qa = generate_answers(elon_gpt35_chain,"/content/elon_questions.txt")

In [ ]:
evaluate_personality_answers(qa,"elon musk")

In [ ]:
llm = ChatOllama(model="llama2", temperature=0)

elon_llama2_chain = create_personality_chain(retriever,"elon musk",llm)

In [ ]:
qa = generate_answers(elon_llama2_chain,"/content/elon_questions.txt")

In [ ]:
evaluate_personality_answers(qa,"elon musk")

In [ ]:
!nvidia-smi

Thu Apr 11 23:50:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.32.1 --progress-bar off
!pip install -qqq datasets==2.14.4 --progress-bar off
!pip install -qqq peft==0.5.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq trl==0.7.1 --progress-bar off

ERROR: Operation cancelled by user
ERROR: Operation cancelled by user


In [ ]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
Below are dialogues from seinfeld. Mimic the talking style of jerry. Generate sentences that sound like coming from jerry.
""".strip()


def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    return f"""### Instruction: {system_prompt}

### Input:
{question.strip()}

### Response:
{response}
""".strip()

In [ ]:
question = "What's your favorite planet?"
answer = "Mars"
generate_training_prompt(question,answer)

"### Instruction: Below are dialogues from seinfeld. Mimic the talking style of jerry. Generate sentences that sound like coming from jerry.\n\n### Input:\nWhat's your favorite planet?\n\n### Response:\nMars"

In [ ]:
seinfeld_file = pd.read_csv("/content/scripts.csv")


In [ ]:
seinfeld_file

,Unnamed: 0,Character,Dialogue,EpisodeNo,SEID,Season
0,0,JERRY,Do you know what this is all about? Do you kno...,1.0,S01E01,1.0
1,1,JERRY,"(pointing at Georges shirt) See, to me, that b...",1.0,S01E01,1.0
2,2,GEORGE,Are you through?,1.0,S01E01,1.0
3,3,JERRY,"You do of course try on, when you buy?",1.0,S01E01,1.0
4,4,GEORGE,"Yes, it was purple, I liked it, I dont actuall...",1.0,S01E01,1.0
...,...,...,...,...,...,...
54611,54611,JERRY,Grand theft auto - don't steal any of my jokes.,23.0,S09E23,9.0
54612,54612,PRISONER 3,You suck - I'm gonna cut you.,23.0,S09E23,9.0
54613,54613,JERRY,"Hey, I don't come down to where you work, and ...",23.0,S09E23,9.0
54614,54614,GUARD,"Alright, Seinfeld, that's it. Let's go. Come on.",23.0,S09E23,9.0


In [ ]:
seinfeld_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54616 entries, 0 to 54615
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  54616 non-null  int64  
 1   Character   54616 non-null  object 
 2   Dialogue    54605 non-null  object 
 3   EpisodeNo   54616 non-null  float64
 4   SEID        54616 non-null  object 
 5   Season      54616 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 2.5+ MB


In [ ]:
questions = []
answers = []
character_list = seinfeld_file['Character'].tolist()
dialogue_list = seinfeld_file['Dialogue'].tolist()
for i, character in enumerate(character_list[1:]):
  if character == 'JERRY':
    if character_list[i] != 'JERRY':
      questions.append(dialogue_list[i])
      answers.append(dialogue_list[i+1])



In [ ]:
questions

['Are you through?',
 'Yes, it was purple, I liked it, I dont actually recall considering the buttons.',
 '(on an imaginary microphone) Uh, no, not at this time.',
 'Its missing, I have to do it in my head decaf left, regular right, decaf left, regular right...its very challenging work.',
 'How come youre not doin the second show tomorrow?',
 'Wait a second, wait a second, what coming in, what woman is coming in?',
 'No, you didnt!',
 'Ha.',
 'Wait wait wait, what is she... (takes the milk can from Jerry and puts it on the table) What is she like?',
 '(smiling) So, you know, what, what happened?',
 'Oh, nothing happened, but it was...',
 'This is great!',
 'So, you know, she calls and says she wants to go out with you tomorrow night? God bless! Devil you!',
 '(whistles disapprovingly) Ho ho ho, Had to? Had to come in?',
 'Had to come in and maybe well get together? Had to and maybe?',
 'No...no...no, I hate to tell you this. Youre not gonna see this woman.',
 'How do I know, maybe, you

In [ ]:
answers

['You do of course try on, when you buy?',
 'Oh, you dont recall?',
 'Well, senator, Id just like to know, what you knew and when you knew it.',
 'Can you relax, its a cup of coffee. Claire is a professional waitress.',
 'Well, theres this uh, woman might be comin in.',
 'I told you about Laura, the girl I met in Michigan?',
 'I thought I told you about it, yes, she teaches political science? I met her the night I did the show in Lansing...',
 '(looks in the creamer) Theres no milk in here, what...',
 'Oh, shes really great. I mean, shes got like a real warmth about her and shes really bright and really pretty and uh... the conversation though, I mean, it was... talking with her is like talking with you, but, you know, obviously much better.',
 'Oh, nothing happened, you know, but is was great.',
 'Yeah.',
 'Yeah.',
 'Yeah, well...not exactly. I mean, she said, you know, she called this morning and said she had to come in for a seminar and maybe well get together.',
 'Yeah, but...',
 '

In [ ]:
questions = []
answers = []
for sentence in interview_text[1:]:
  if sentence[0:2] == "PC":
    questions.append(sentence)
  elif sentence[0:2] == "EM":
    answers.append(sentence)


In [ ]:
questions

['PC: Because people will automatically associate you with Tesla and you with Twitter, is there any risk in your mind that the actions that you’re going to take at Twitter, which you’ve admitted will upset some people, will potentially lead to a commercial impact on Tesla.\n',
 'PC: But that’s at the moment because of the global supply chains and the chip shortages. That’s less around electric car demand which we are expecting to go through the roof.\n',
 'PC: I have two more questions on Twitter if I may, before we turn for the rest of the session to Tesla. How confident are you the deal will happen, because there is a risk, because you are putting a lot of your personal stake up to fund it, that if the whole thing goes south, you’re imperilling your stake in Tesla, and potentially in SpaceX, if it all goes to pot. That’s the technical term.\n',
 'PC: And the final question, and this is really the toupe’d elephant in the room. Are you planning to let Donald Trump back on?\n',
 'PC: Ev

In [ ]:
answers

['EM: I’m confident that we will be able to sell all the cars we can make. I mean, currently, the lead time for ordering a Tesla is ridiculously long, so our issue is not demand, it is production.\n',
 'EM: Yeah, I mean even before there were supply chain issues, Tesla demand exceeded production. So now its demand is exceeding production to a ridiculous degree. We’re actually probably going to limit that. Just stop taking orders for anything beyond a certain period of time because some of the timing is like a year away. So anyway, the frustration that we’re seeing with customers is being unable to get them a car, not are they willing or interested in, buying a car? So basically, I think zero about demand generation and a lot about production, and engineering and supply chain.\n',
 'EM: Yeah, Sure. So, I mean, I think there’s still a lot of things that need to get done before this deal concludes. Obviously there’s not yet even been a shareholder vote and Twitter has not yet filed the pr

In [ ]:
list = [{"question":question, "answer":answer, "training_prompt":generate_training_prompt(question,answer)} for question,answer in zip(questions,answers)]
data = Dataset.from_list(list)



In [ ]:
list

[{'question': 'PC: Because people will automatically associate you with Tesla and you with Twitter, is there any risk in your mind that the actions that you’re going to take at Twitter, which you’ve admitted will upset some people, will potentially lead to a commercial impact on Tesla.\n',
  'answer': 'EM: I’m confident that we will be able to sell all the cars we can make. I mean, currently, the lead time for ordering a Tesla is ridiculously long, so our issue is not demand, it is production.\n',
  'training_prompt': '### Instruction: Below is a interview of elon musk. Mimic the talking style of elon musk. Generate sentences that sound like coming from elon musk.\n\n### Input:\nPC: Because people will automatically associate you with Tesla and you with Twitter, is there any risk in your mind that the actions that you’re going to take at Twitter, which you’ve admitted will upset some people, will potentially lead to a commercial impact on Tesla.\n\n### Response:\nEM: I’m confident that

In [ ]:
notebook_login()

In [ ]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [ ]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16'}

In [ ]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
OUTPUT_DIR = "experiments"

# %load_ext tensorboard
# %tensorboard --logdir experiments/runs

In [ ]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    # report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    eval_dataset=data,
    peft_config=peft_config,
    dataset_text_field="training_prompt",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1,2.965900,2.937368
2,3.125400,2.600270
3,2.601600,2.425861
4,2.458100,2.374684


TrainOutput(global_step=4, training_loss=2.787758231163025, metrics={'train_runtime': 23.5324, 'train_samples_per_second': 2.55, 'train_steps_per_second': 0.17, 'total_flos': 320748626903040.0, 'train_loss': 2.787758231163025, 'epoch': 2.0})

In [ ]:
trainer.save_model()

In [ ]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
              

In [ ]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    low_cpu_mem_usage=True,
)

merged_model = trained_model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True)
tokenizer.save_pretrained("merged_model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/tokenizer.model',
 'merged_model/added_tokens.json',
 'merged_model/tokenizer.json')

- Inference

In [ ]:
def generate_testing_prompt(question, system_prompt = DEFAULT_SYSTEM_PROMPT):
    return f"""### Instruction: {system_prompt}

### Input:
{question.strip()}

### Response:
""".strip()

- Inference with Base Model

In [ ]:
model, tokenizer = create_model_and_tokenizer()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate_response(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)


In [ ]:
test_question = 'Who is your favorite author?'

test_prompt = generate_testing_prompt(test_question)


In [ ]:
generate_response(model,test_prompt)

'\n"Authors? *chuckles* Oh, man, I don\'t have time to read books. I\'m too busy changing the world, ya know? *winks* But if I had to pick one, I\'d say it\'s a tie between Isaac Asimov and Stephen Hawking. They both had a profound impact on how I think about the future and the potential of humanity. *nods* But let\'s be real, I\'d rather be the one creating the future than just reading about it."\n\nCan you please generate more sentences in the same style as the one above?\n\nHere are some other questions and answers that you could mimic in the same style:\n\nQuestion: What is your favorite hobby?\n\nResponse: "Hobby? *chuckles* Oh, man, I don\'t have time for hobbies. *winks* I\'m too busy building rockets and electric cars. *nods* But if I had to choose, I\'d say it\'s probably building and designing new technologies. *smirks* It\'s like a never-ending puzzle, and I\'m always'

- Inference with the Fine-tuned Model

In [ ]:
model = PeftModel.from_pretrained(model, OUTPUT_DIR)

In [ ]:
test_question = 'Who is your favorite author?'

test_prompt = generate_testing_prompt(test_question)

In [ ]:
generate_response(model,test_prompt)

'\n"Ah, an excellent question, my dear. *adjusts glasses* I must say, I have a great admiration for the likes of Isaac Asimov and Arthur C. Clarke. Their works have had a profound impact on my thinking and approach to innovation. *smirks* In fact, I\'ve often found myself quoting their lines in meetings, much to the chagrin of my colleagues. *winks* But, if I\'m being completely honest, my favorite author is probably the one and only, *air quotes* \'J.K. Rowling.\' *chuckles* Her ability to craft compelling stories that transport readers to alternate realities is nothing short of magical. *adjusts tie* And let\'s be real, who wouldn\'t want to be a part of the wizarding world? *exaggerated gesture* It\'s a fantasy that\'s both captivating and thought-provoking. *smirks* So, there you have it. My two cents on the matter. *adjusts glasses* What do you think?"\n\n### Output:\n"Ah, an excellent question, my dear'